# Importing the libraries
In this part, we are importing the libraries we need.

In [34]:
# Importing files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import sys
import glob
import datetime
import calendar
import csv

# Spark imports
from pyspark.rdd import RDD
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
from pyspark.sql.functions import col, asc, desc, split

# used to check if the array has a value
from pyspark.sql.functions import array_contains

# used for the encoding of categories
from sklearn.preprocessing import MultiLabelBinarizer

# Spark 
In this project, we are using apache spark as well as Pandas.

In [4]:
# Initialize a spark session.
def init_spark():
    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

In [5]:
spark = init_spark() # Initializate spark

# Buisiness dataset 
The first dataset is buisiness dataset. We are importing the dataset to work with the features and extract user and profile items.

In [26]:
df_business = pd.read_json('D:\Msc CS\Big Data\Project\Data\yelp_academic_dataset_business.json',lines=True)
df_business

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,53.468419,-113.492054,3.0,13,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","Nail Salons, Beauty & Spas","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3..."
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Shopping, Jewelry, Piercing, Toy Stores, Beaut...",None
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ..."


## Filtering restaurant businesses
Now we want to filter the Restaurant category from the dataset

In [37]:
df_business = df_business[df_business['categories'].str.contains('Restaurants',na=False)]
df_business

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr...",None
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '..."
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150325,l9eLGG9ZKpLJzboZq-9LRQ,Wawa,19 N Bishop Ave,Clifton Heights,PA,19018,39.925656,-75.310344,3.0,11,1,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Restaurants, Sandwiches, Convenience Stores, C...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
150327,cM6V90ExQD6KMSU3rRB5ZA,Dutch Bros Coffee,1181 N Milwaukee St,Boise,ID,83704,43.615401,-116.284689,4.0,33,1,"{'WiFi': ''free'', 'RestaurantsGoodForGroups':...","Cafes, Juice Bars & Smoothies, Coffee & Tea, R...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-17:0', '..."
150336,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,1108 S 9th St,Philadelphia,PA,19147,39.935982,-75.158665,4.5,35,1,"{'WheelchairAccessible': 'False', 'Restaurants...","Restaurants, Mexican","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
150339,2O2K6SXPWv56amqxCECd4w,The Plum Pit,4405 Pennell Rd,Aston,DE,19014,39.856185,-75.427725,4.5,14,1,"{'RestaurantsDelivery': 'False', 'BusinessAcce...","Restaurants, Comfort Food, Food, Food Trucks, ...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."


In [42]:
data = df_business['categories'].notnull()
data

74851     True
123532    True
96854     True
19270     True
26000     True
          ... 
67916     True
131766    True
55063     True
18422     True
78503     True
Name: categories, Length: 52268, dtype: bool

## Now we want to encode the field called categories

In [46]:
mlb = MultiLabelBinarizer()

#create boolean mask matched non NaNs values
mask = df_business['categories'].notnull()

#filter by boolean indexing
arr = mlb.fit_transform(df_business.loc[mask, 'categories'].dropna().str.strip('[]').str.split(','))

#create DataFrame and add missing (NaN)s index values
df_categories = (pd.DataFrame(arr, index=df_business.index[mask], columns=mlb.classes_)
               .reindex(df_business.index, fill_value=0))

df_categories.sort_index()

,Acai Bowls,Accessories,Accountants,Active Life,Acupuncture,Adult,Adult Entertainment,Advertising,Afghan,African,...,Wholesale Stores,Wholesalers,Wigs,Wine Bars,Wine Tasting Room,Wine Tours,Wineries,Women's Clothing,Wraps,Yoga
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150325,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
150327,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
150336,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
150339,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Preprocessing the cities: 
After the preprocessing, the number of unique cities dropped. 

In [38]:
df_business = df_business.sort_values(by=['city'])  # Sorting by name
print('Original unique city values:', len(df_business['city'].unique()))  # Printing the unique values
df_business['city'] = df_business['city'].str.lower()
print('Unique city values after changing all letter to lowercase:', len(df_business['city'].unique())) # Printing the unique values

Original unique city values: 920
Unique city values after changing all letter to lowercase: 859


### Encoding the city and state

In [48]:
# Encoding feature City and State features
df_city = pd.get_dummies(df_business.city).sort_index()
df_state = pd.get_dummies(df_business.state).sort_index()
df_city # printing for state (example)

,abington,abington township,affton,aldan,algiers,alloway,alton,ambler,antioch,apollo beach,...,wycombe,wyncote,wyndlake condominium,wyndmoor,wynnewood,yardley,yeadon,zephyrhills,zieglerville,zionsville
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150325,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
150327,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
150336,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
150339,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
df_restData = df_business[['business_id', 'latitude','longitude','stars','is_open']].sort_index()

In [72]:
frames = [df_restData, df_categories, df_city, df_state]
df_itemProf = pd.concat(frames, axis=1)
df_itemProf.sort_index()

,business_id,latitude,longitude,stars,is_open,Acai Bowls,Accessories,Accountants,Active Life,Acupuncture,...,IN,LA,MO,MT,NC,NJ,NV,PA,TN,XMS
3,MTSW4McQd7CbVtyjqoe9mw,39.955505,-75.155564,4.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,CF33F8-E6oudUQ46HnavjQ,36.269593,-87.058943,2.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8,k0hlBqXX-Bt0vf1op7Jr1w,38.565165,-90.321087,3.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9,bBDDEgkFA1Otx9Lfe7BZUQ,36.208102,-86.768170,1.5,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
11,eEOYSgkmpB90uNA7lDOMRA,27.955269,-82.456320,4.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150325,l9eLGG9ZKpLJzboZq-9LRQ,39.925656,-75.310344,3.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
150327,cM6V90ExQD6KMSU3rRB5ZA,43.615401,-116.284689,4.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
150336,WnT9NIzQgLlILjPT0kEcsQ,39.935982,-75.158665,4.5,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
150339,2O2K6SXPWv56amqxCECd4w,39.856185,-75.427725,4.5,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
# apply normalization technique to Column 1
df_itemProf_normalized = df_itemProf.copy()
df_itemProf_normalized['latitude'] = (df_itemProf['latitude'] - df_itemProf['latitude'].mean()) / df_itemProf['latitude'].std()    
df_itemProf_normalized['longitude'] = (df_itemProf['longitude'] - df_itemProf['longitude'].mean()) / df_itemProf['longitude'].std()      
# view normalized data  
display(df_itemProf_normalized.sort_index())

,business_id,latitude,longitude,stars,is_open,Acai Bowls,Accessories,Accountants,Active Life,Acupuncture,...,IN,LA,MO,MT,NC,NJ,NV,PA,TN,XMS
3,MTSW4McQd7CbVtyjqoe9mw,0.492047,0.918595,4.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,CF33F8-E6oudUQ46HnavjQ,-0.121110,0.056730,2.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8,k0hlBqXX-Bt0vf1op7Jr1w,0.260762,-0.179465,3.0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9,bBDDEgkFA1Otx9Lfe7BZUQ,-0.131339,0.077784,1.5,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
11,eEOYSgkmpB90uNA7lDOMRA,-1.504208,0.389983,4.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150325,l9eLGG9ZKpLJzboZq-9LRQ,0.487081,0.907388,3.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
150327,cM6V90ExQD6KMSU3rRB5ZA,1.100875,-2.059361,4.0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
150336,WnT9NIzQgLlILjPT0kEcsQ,0.488799,0.918370,4.5,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
150339,2O2K6SXPWv56amqxCECd4w,0.475525,0.898889,4.5,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
